# 自動生成ツール使用例


## 必要なパッケージをインポート


In [ ]:
import os
import json

import requests
from autogen_agentchat.agents import AssistantAgent
from autogen_core.models import UserMessage
from autogen_ext.models.azure import AzureAIChatCompletionClient
from azure.core.credentials import AzureKeyCredential
from autogen_core import CancellationToken
from autogen_core.tools import FunctionTool
from autogen_agentchat.messages import TextMessage
from autogen_agentchat.ui import Console
from typing import Any, Callable, Set, Dict, List, Optional

## クライアントの作成

このサンプルでは、LLMにアクセスするために[GitHub Models](https://aka.ms/ai-agents-beginners/github-models)を使用します。

`model` は `gpt-4o-mini` として定義されています。GitHub Modelsマーケットプレイスで利用可能な別のモデルに変更して、異なる結果を試してみてください。

簡単なテストとして、シンプルなプロンプト `What is the capital of France` を実行してみましょう。


In [ ]:
client = AzureAIChatCompletionClient(
    model="gpt-4o-mini",
    endpoint="https://models.inference.ai.azure.com",
    # To authenticate with the model you will need to generate a personal access token (PAT) in your GitHub settings.
    # Create your PAT token by following instructions here: https://docs.github.com/en/authentication/keeping-your-account-and-data-secure/managing-your-personal-access-tokens
    credential=AzureKeyCredential(os.environ["GITHUB_TOKEN"]),
    model_info={
        "json_output": True,
        "function_calling": True,
        "vision": True,
        "family": "unknown",
    },
)

result = await client.create([UserMessage(content="What is the capital of France?", source="user")])
print(result)

## 関数の定義

この例では、エージェントに利用可能な休暇先のリストとその空き状況を提供するツールとして機能する関数へのアクセスを与えます。

これは、例えば旅行代理店が旅行データベースにアクセスできるシナリオだと考えることができます。

このサンプルを進める中で、エージェントが呼び出せる新しい関数やツールを自由に定義してみてください。


In [ ]:
from typing import Dict, List, Optional


def vacation_destinations(city: str) -> tuple[str, str]:
    """
    Checks if a specific vacation destination is available
    
    Args:
        city (str): Name of the city to check
        
    Returns:
        tuple: Contains city name and availability status ('Available' or 'Unavailable')
    """
    destinations = {
        "Barcelona": "Available",
        "Tokyo": "Unavailable",
        "Cape Town": "Available",
        "Vancouver": "Available",
        "Dubai": "Unavailable",
    }

    if city in destinations:
        return city, destinations[city]
    else:
        return city, "City not found"

# Example usage:
# city, status = vacation_destinations("Barcelona")
# print(f"How about visiting {city}? It's currently {status} there!")

## 関数ツールの定義  
エージェントが `vacation_destinations` を `FunctionTool` として使用できるようにするには、それをそのように定義する必要があります。  

また、ユーザーがリクエストしたタスクに関連してそのツールが何に使われるかをエージェントが識別するのに役立つ説明も提供します。


In [ ]:
get_vacations = FunctionTool(
    vacation_destinations, description="Search for vacation destinations and if they are available or not."
)

## エージェントの定義

以下のコードでエージェントを作成します。`system_message` を定義して、ユーザーが旅行先を見つけるのを手助けするための指示をエージェントに与えます。

また、`reflect_on_tool_use` パラメータを true に設定します。これにより、ツールの呼び出し結果を LLM が受け取り、それを自然な言葉で応答することが可能になります。

このパラメータを false に設定すると、その違いを確認することができます。


In [ ]:
agent = AssistantAgent(
    name="assistant",
    model_client=client,
    tools=[get_vacations],
    system_message="You are a travel agent that helps users find vacation destinations.",
    reflect_on_tool_use=True,
)

## エージェントの実行

これで、東京旅行をお願いする初回のユーザーメッセージを使ってエージェントを実行できます。

都市の目的地を変更して、その都市の利用可能性に対するエージェントの応答を確認することができます。


In [ ]:
async def assistant_run() -> None:
    response = await agent.on_messages(
        [TextMessage(content="I would like to take a trip to Tokyo", source="user")],
        cancellation_token=CancellationToken(),
    )
    print(response.inner_messages)
    print(response.chat_message)


# Use asyncio.run(assistant_run()) when running in a script.
await assistant_run()


---

**免責事項**:  
この文書はAI翻訳サービス[Co-op Translator](https://github.com/Azure/co-op-translator)を使用して翻訳されています。正確性を追求しておりますが、自動翻訳には誤りや不正確な部分が含まれる可能性があります。元の言語で記載された文書が正式な情報源とみなされるべきです。重要な情報については、専門の人間による翻訳を推奨します。この翻訳の使用に起因する誤解や誤解について、当社は責任を負いません。
